<a href="https://colab.research.google.com/github/nafiul-araf/Anomaly-Detection/blob/main/Model_Based_Outlier_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Reference:*** https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/

In [103]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [104]:
df=pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [105]:
data=df.values
data

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 3.9690e+02, 4.9800e+00,
        2.4000e+01],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 3.9690e+02, 9.1400e+00,
        2.1600e+01],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 3.9283e+02, 4.0300e+00,
        3.4700e+01],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 5.6400e+00,
        2.3900e+01],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 3.9345e+02, 6.4800e+00,
        2.2000e+01],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 7.8800e+00,
        1.1900e+01]])

In [106]:
df.shape

(506, 14)

In [107]:
X, y=data[:, :-1], data[:, -1]

In [108]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.33, random_state=np.random.seed(10))

In [109]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((339, 13), (167, 13), (339,), (167,))

###Baseline Model

In [110]:
model=LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [111]:
y_hat=model.predict(X_test)
print('Mean Squared Error Baseline for Model: %.3f'%mean_squared_error(y_test, y_hat))

Mean Squared Error Baseline for Model: 27.101


###Outlier Detection with IForest

In [112]:
from sklearn.ensemble import IsolationForest

In [113]:
#identify outliers in the training dataset
iso=IsolationForest(contamination=0.1)
y_hat=iso.fit_predict(X_train)

In [114]:
print(y_hat)

[ 1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1
  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1 -1  1  1 -1  1  1  1  1  1 -1 -1  1 -1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1 -1  1  1 -1 -1  1 -1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1
 -1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1 -1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1
  1 -1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1  1

In [115]:
#select all rows that are not outliers
mask=y_hat != -1
X_train_iso, y_train_iso=X_train[mask, :], y_train[mask]
X_train_iso.shape, y_train_iso.shape

((305, 13), (305,))

***Removed 34 outliers***

In [116]:
model=LinearRegression()
model.fit(X_train_iso, y_train_iso)

y_hat=model.predict(X_test)
print('Mean Squared Error with Isoforest Outlier Detector: %.3f'%mean_squared_error(y_test, y_hat))

Mean Squared Error with Isoforest Outlier Detector: 28.586


***Not performing well***

###Minimum Coviance Determinant

In [117]:
from sklearn.covariance import EllipticEnvelope

In [118]:
ee=EllipticEnvelope(contamination=0.1)
y_hat=ee.fit_predict(X_train)
print(y_hat)

[ 1  1  1  1  1  1  1  1  1 -1  1  1 -1  1  1  1  1 -1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1 -1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1 -1 -1  1  1
  1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1
  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1 -1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1 -1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1  1  1  1  1  1
  1  1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1

In [119]:
mask=y_hat != -1
X_train_ee, y_train_ee=X_train[mask, :], y_train[mask]
X_train_ee.shape, y_train_ee.shape

((305, 13), (305,))

***Removed 34 outliers***

In [120]:
model=LinearRegression()
model.fit(X_train_ee, y_train_ee)

y_hat=model.predict(X_test)
print('Mean Squared Error with Minimum Variance Determinant Outlier Detection Method: %.3f'%mean_squared_error(y_test, y_hat))

Mean Squared Error with Minimum Variance Determinant Outlier Detection Method: 24.945


***Far Better than Isoforest Method and Performs well***

###Local Outlier Factor

In [121]:
from sklearn.neighbors import LocalOutlierFactor

In [122]:
lof=LocalOutlierFactor(contamination=0.1)
y_hat=lof.fit_predict(X_train)
print(y_hat)

[ 1  1  1  1  1  1  1  1 -1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1
  1  1 -1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1 -1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1
  1  1  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1 -1  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1 -1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1 -1 -1  1
  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1 -1  1  1  1  1  1  1  1 -1  1  1  1  1 -1

In [123]:
mask=y_hat != -1
X_train_lof, y_train_lof=X_train[mask, :], y_train[mask]
X_train_lof.shape, y_train_lof.shape

((305, 13), (305,))

***Removed 34 Outliers***

In [124]:
model=LinearRegression()
model.fit(X_train_lof, y_train_lof)

y_hat=model.predict(X_test)
print('Mean Squared Error with Local Outlier Factor Method: %.3f'%mean_squared_error(y_test, y_hat))

Mean Squared Error with Local Outlier Factor Method: 27.592


***Better than Isoforest Method***

###One-Class SVM

In [125]:
from sklearn.svm import OneClassSVM

In [126]:
ocs=OneClassSVM(nu=0.1)
y_hat=ocs.fit_predict(X_train)
print(y_hat)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1 -1  1  1  1
  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1 -1  1  1  1
  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1 -1 -1  1  1
  1  1  1 -1  1 -1  1  1  1  1 -1  1 -1  1  1  1  1  1  1 -1  1  1  1  1
 -1  1  1 -1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1 -1  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1 -1  1  1  1  1  1  1  1 -1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1 -1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1
 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

In [127]:
mask=y_hat!=-1
X_train_ocs, y_train_ocs=X_train[mask, :], y_train[mask]
X_train_ocs.shape, y_train_ocs.shape

((305, 13), (305,))

***Removed 34 Outliers***

In [128]:
model=LinearRegression()
model.fit(X_train_ocs, y_train_ocs)

y_hat=model.predict(X_test)
print('Mean Squared Error with One Class SVM Outlier Detection Method: %.3f'%mean_squared_error(y_test, y_hat))

Mean Squared Error with One Class SVM Outlier Detection Method: 27.709


***Better than Isoforest but not Overall***